## Analysis of the encoding of a Nuclear Shell Hamiltonian to a qubit Hamiltonian using OpenFermion

#### Imports

In [1]:
from collections import Counter
from src.hamiltonian_utils import get_twobody_nuclearshell_model,FermiHubbardHamiltonian,SingleParticleState
import numpy as np
import torch
from typing import Dict
from src.qml_models import AdaptVQEFermiHubbard
from src.qml_utils.train import Fit
from src.qml_utils.utils import configuration
from scipy.sparse.linalg import eigsh,expm_multiply
from tqdm import trange
import matplotlib.pyplot as plt
import scipy


from openfermion.ops import FermionOperator
from openfermion.transforms import jordan_wigner
import cirq
from openfermion.utils import count_qubits
from openfermion.circuits import trotterize_exp_qubop_to_qasm
def plot_spectrum(eigenvalues):
    """
    Plot the vertical spectrum of a Hamiltonian, showing the eigenvalues as horizontal lines 
    and indicating their degeneracy.

    Parameters:
    eigenvalues (array-like): Array of eigenvalues of the Hamiltonian.
    """
    # Count the degeneracy of each eigenvalue
    degeneracy = Counter(eigenvalues)

    # Prepare data for plotting
    unique_eigenvalues = list(degeneracy.keys())
    degeneracies = list(degeneracy.values())

    # Plot the spectrum
    plt.figure(figsize=(6, 10))
    for i, (eig, deg) in enumerate(zip(unique_eigenvalues, degeneracies)):
        plt.hlines(eig, i - 0.2 * deg, i + 0.2 * deg, colors='b', linewidth=5)
        plt.text(i, eig, f'{deg}', horizontalalignment='center', verticalalignment='bottom', fontsize=24, color='r')

    # Make the plot fancy
    plt.title('Spectrum of the Hamiltonian', fontsize=16)
    plt.ylabel('Eigenvalue', fontsize=14)
    plt.xlabel('Index (degeneracy indicated by text)', fontsize=14)
    plt.xticks(range(len(unique_eigenvalues)), ['']*len(unique_eigenvalues))  # Remove x-axis ticks
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()

    # Show the plot
    plt.show()

file_name='data/cki'
SPS=SingleParticleState(file_name=file_name)
energies=SPS.energies

/opt/anaconda3/envs/dft_env/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


##### Hyperparameters

In [2]:
twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

habcd=np.zeros((energies.shape[0],energies.shape[0],energies.shape[0],energies.shape[0]))
iso_dict={-0.5:'n',0.5:'p'}
values=np.asarray(list(twobody_matrix.values()))
print(np.average(np.abs(values)))
for key in twobody_matrix.keys():
    i,j,k,l=key
    habcd[i,j,k,l]=twobody_matrix[key]
    (n,_,ja,ma,_,tza)=SPS.state_encoding[i]
    (n,_,jb,mb,_,tzb)=SPS.state_encoding[j]
    (n,_,jc,mc,_,tzc)=SPS.state_encoding[k]
    (n,_,jd,md,_,tzd)=SPS.state_encoding[l]

    print(ja,ma,iso_dict[tza]+'+'+iso_dict[tzb],jb,mb,'-->',jc,mc,iso_dict[tzc]+'+'+iso_dict[tzd],jd,md)
    print('cross section=',twobody_matrix[key],'\n')
    


average_unit_energy=np.average(np.abs(np.asarray(list(twobody_matrix.values()))))

Computing the matrix, pls wait... (u_u) 



100%|██████████| 12/12 [00:02<00:00,  5.49it/s]


1.4069726715197104
1.5 -1.5 p+p 1.5 -0.5 --> 1.5 -1.5 p+p 1.5 -0.5
cross section= -0.6490000000000002 

1.5 -1.5 p+p 1.5 -0.5 --> 1.5 -0.5 p+p 1.5 -1.5
cross section= 0.6490000000000002 

1.5 -0.5 p+p 1.5 -1.5 --> 1.5 -1.5 p+p 1.5 -0.5
cross section= 0.6490000000000002 

1.5 -0.5 p+p 1.5 -1.5 --> 1.5 -0.5 p+p 1.5 -1.5
cross section= -0.6490000000000002 

1.5 -1.5 p+p 1.5 -0.5 --> 1.5 -1.5 p+p 0.5 -0.5
cross section= 2.2055000000000007 

1.5 -1.5 p+p 1.5 -0.5 --> 0.5 -0.5 p+p 1.5 -1.5
cross section= -2.2055000000000007 

1.5 -0.5 p+p 1.5 -1.5 --> 1.5 -1.5 p+p 0.5 -0.5
cross section= -2.2055000000000007 

1.5 -0.5 p+p 1.5 -1.5 --> 0.5 -0.5 p+p 1.5 -1.5
cross section= 2.2055000000000007 

1.5 -1.5 p+p 1.5 0.5 --> 1.5 -1.5 p+p 1.5 0.5
cross section= -0.6490000000000002 

1.5 -1.5 p+p 1.5 0.5 --> 1.5 0.5 p+p 1.5 -1.5
cross section= 0.6490000000000002 

1.5 0.5 p+p 1.5 -1.5 --> 1.5 -1.5 p+p 1.5 0.5
cross section= 0.6490000000000002 

1.5 0.5 p+p 1.5 -1.5 --> 1.5 0.5 p+p 1.5 -1.5
cross sectio

##### Build up the Hamiltonian in fermionic representation

In [61]:
# Initialize the Hamiltonian
hamiltonian = FermionOperator()


print(SPS.state_encoding)

# Add on-site potential terms
for i, potential in enumerate(energies):
    hamiltonian += FermionOperator(((i, 1), (i, 0)), potential)

# Add two-body interaction terms
for (p, q, r, s), interaction in twobody_matrix.items():
    if q==r:
        
        hamiltonian += FermionOperator(((p, 1), (s, 0)), 0.25*interaction)-FermionOperator(((p, 1), (r, 1), (q, 0), (s, 0)), 0.25*interaction)

    else:
        hamiltonian -= FermionOperator(((p, 1), (r, 1), (q, 0), (s, 0)), 0.25*interaction)


print("Fermion Hamiltonian:")
print(hamiltonian)

[(2, 0, 2.5, -2.5, 0.5, 0.5), (2, 0, 2.5, -1.5, 0.5, 0.5), (2, 0, 2.5, -0.5, 0.5, 0.5), (2, 0, 2.5, 0.5, 0.5, 0.5), (2, 0, 2.5, 1.5, 0.5, 0.5), (2, 0, 2.5, 2.5, 0.5, 0.5), (1, 0, 0.5, -0.5, 0.5, 0.5), (1, 0, 0.5, 0.5, 0.5, 0.5), (2, 0, 1.5, -1.5, 0.5, 0.5), (2, 0, 1.5, -0.5, 0.5, 0.5), (2, 0, 1.5, 0.5, 0.5, 0.5), (2, 0, 1.5, 1.5, 0.5, 0.5), (2, 0, 2.5, -2.5, 0.5, -0.5), (2, 0, 2.5, -1.5, 0.5, -0.5), (2, 0, 2.5, -0.5, 0.5, -0.5), (2, 0, 2.5, 0.5, 0.5, -0.5), (2, 0, 2.5, 1.5, 0.5, -0.5), (2, 0, 2.5, 2.5, 0.5, -0.5), (1, 0, 0.5, -0.5, 0.5, -0.5), (1, 0, 0.5, 0.5, 0.5, -0.5), (2, 0, 1.5, -1.5, 0.5, -0.5), (2, 0, 1.5, -0.5, 0.5, -0.5), (2, 0, 1.5, 0.5, 0.5, -0.5), (2, 0, 1.5, 1.5, 0.5, -0.5)]
Fermion Hamiltonian:
-3.9257 [0^ 0] +
-0.05172500000000001 [0^ 1^ 0 1] +
0.3337250000000001 [0^ 1^ 0 8] +
0.05172500000000001 [0^ 1^ 1 0] +
-0.3337250000000001 [0^ 1^ 8 0] +
-0.05172500000000001 [0^ 2^ 0 2] +
0.20436399110257913 [0^ 2^ 0 9] +
0.26383277803492317 [0^ 2^ 1 8] +
0.05172500000000001 [0^ 2^

#### Encode to Qubit via Jordan-Wigner

In [62]:
from openfermion.transforms import jordan_wigner

# Transform to qubit representation
jw_hamiltonian = jordan_wigner(hamiltonian)
print("\nJordan-Wigner Transformed Hamiltonian:")
print(jw_hamiltonian)


Jordan-Wigner Transformed Hamiltonian:
(53.73737500000004+0j) [] +
(-0.047544395371589304+0j) [X0 X1 X2 X3] +
(-0.08524182168432642+0j) [X0 X1 X2 Z3 Z4 Z5 Z6 X7] +
(-0.13191638901746158+0j) [X0 X1 X2 Z3 Z4 Z5 Z6 Z7 X8] +
(-0.01786126007052842+0j) [X0 X1 X2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
(-0.047544395371589304+0j) [X0 X1 Y2 Y3] +
(-0.08524182168432642+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Y7] +
(-0.13191638901746158+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Y8] +
(-0.01786126007052842+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Y10] +
(-0.04482528583288678+0j) [X0 X1 X3 X4] +
(-0.02841394056144215+0j) [X0 X1 X3 Z4 Z5 X6] +
(-0.08806946942344042+0j) [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 X9] +
(-0.010312203309785493+0j) [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
(-0.04482528583288678+0j) [X0 X1 Y3 Y4] +
(-0.02841394056144215+0j) [X0 X1 Y3 Z4 Z5 Y6] +
(-0.08806946942344042+0j) [X0 X1 Y3 Z4 Z5 Z6 Z7 Z8 Y9] +
(-0.010312203309785493+0j) [X0 X1 Y3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
(-0.08622499999999997+0j) [X0 X1 X4 X5] +
(0.080366760204909+0j) [X0 X

##### Encode to Qubit via Bravi-Kitaev

In [63]:
from openfermion.transforms import bravyi_kitaev

# Transform to qubit representation
bk_hamiltonian = bravyi_kitaev(hamiltonian)
print("\nBravyi Kitaev Transformed Hamiltonian:")
print(bk_hamiltonian)


Bravyi Kitaev Transformed Hamiltonian:
(53.73737500000004+0j) [] +
(0.08598714433456602+0j) [X0 X1 X2 X3 Y7 Y8 X9 X11] +
(-0.050787499999999944+0j) [X0 X1 X2 Y3 Y5] +
(0.08598714433456602+0j) [X0 X1 Y2 X3 Y7 X8 X9 X11] +
(0.050787499999999944+0j) [X0 X1 Y2 Y3 Z4 X5] +
(-0.08111559418839094+0j) [X0 X1 X3 X4 Y5 X6 Y7 Z8 X9 X11] +
(0.08111559418839094+0j) [X0 X1 X3 X4 Y5 Y6 Y7 Y9 X11] +
(-0.04683211014014395+0j) [X0 X1 X3 X4 Y5 Z6 Y7 X8 X9 X11] +
(-0.025410599923175462+0j) [X0 X1 X3 X4 Z5 Y7 Y8 X9 X11] +
(0.025410599923175462+0j) [X0 X1 X3 X4 Y7 Y8 X9 X11] +
(-0.08111559418839094+0j) [X0 X1 X3 Y4 Y5 X6 Y7 Y9 X11] +
(-0.08111559418839094+0j) [X0 X1 X3 Y4 Y5 Y6 Y7 Z8 X9 X11] +
(-0.04683211014014395+0j) [X0 X1 X3 Y4 Y5 Z6 Y7 Y8 X9 X11] +
(-0.025410599923175462+0j) [X0 X1 X3 Y4 Z5 Y7 X8 X9 X11] +
(0.025410599923175462+0j) [X0 X1 X3 Y4 Y7 X8 X9 X11] +
(-0.060459994213509664+0j) [X0 X1 X3 Z4 Y5 X6 Y7 Z9 X10 X11] +
(0.060459994213509664+0j) [X0 X1 X3 Z4 Y5 Y6 Y7 Z9 Y10 X11] +
(0.112597736248706

##### Count the CNOT gates

In [66]:
from cirq.contrib.qasm_import import circuit_from_qasm
# Define qubits
exp_h_qasm=trotterize_exp_qubop_to_qasm(
    bk_hamiltonian,
    evolution_time=1,
    trotter_number=1,
    trotter_order=1,
    term_ordering=None,
    k_exp=1.0,
    qubit_list=None,
    ancilla=None
)
qasm_str = ''.join(exp_h_qasm)


qasm_list=qasm_str.split()

count=0
for str in qasm_list:
    if str[-4:]=='CNOT':
        count+=1
    
print(count)
# print(ops_in_circuit[2])
# for i in range(len(ops_in_circuit)):

#     print(op[:3])
#     if 'CNOT'==op[:3]:
#         count+=1

# print(count)    
#circuit=circuit_from_qasm(qasm_str)
#print(circuit)


198210
